## Bayesian Structural Time Series Model for Stock Price Prediction

#### Team Members:
- Asteria Xu (56376452)
- Yufei Shen (29116514)

#### Introduction:
Financial time series forecasting plays an important role in supporting data-driven decision making for traders. Due to the inherent non-stationarity and volatility in financial markets, stock price prediction is one of the most challenging tasks in this area. In this project, we aim to forecast the closing price of TESLA using the TSLA stock dataset (https://www.kaggle.com/datasets/varpit94/tesla-stock-data-updated-till-28jun2021), which is obtained from Kaggle and contains 7 variables related to stock from 2010 to 2022. The stock is an ideal test case for assessing the flexibility and robustness of Bayesian methods versus frequentist methods.

Our analysis will only focus on the most recent 12-month period to evaluate the short-term forecasting performance of  the Bayesian Structural Time Series (BSTS) model and the Autoregressive Integrated Moving Average (ARIMA) model. The ARIMA model is the cornerstone of classical time series analysis. It models time dependence through differencing and autocorrelation modelling, which is particularly effective for static processes. However, its linearity and constant variance assumptions can limit its effectiveness when applied to highly volatile financial data. On the other hand, the BSTS model employs a Bayesian state-space framework to decompose the observed series into latent components such as trend and seasonality effects, which is beneficial when combining prior knowledge and explicitly modeling uncertainty. It has been used in several real-world applications, including Google's forecasting tool, and is known for generating well-calibrated forecasts, even in the presence of model misspecification. This study contributes to the broader discussion of Bayesian and frequency methods in time series modeling by comparing the forecasting performance and uncertainty quantification of these two models in the dynamic real world. 

#### Appendix:

In [ ]:
library(dplyr)
library(readr)
library(lubridate)
library(bsts)
library(forecast)
library(ggplot2)
library(PerformanceAnalytics)

# Load and preprocess data 
tsla <- read_csv("TSLA.csv") %>%
  mutate(Date = as_date(Date),
         Close = as.numeric(Close)) %>%
  filter(Date >= max(Date) - months(12)) %>%
  arrange(Date)

y <- tsla$Close
dates <- tsla$Date

# Split into training and testing
train_size <- floor(0.9 * length(y))
train <- window(ts(y, frequency = 252), end = c(1, train_size))
test <- window(ts(y, frequency = 252), start = c(1, train_size + 1))

# Bayesian Structural Time Series (BSTS)
# Set priors
level_prior <- SdPrior(sigma.guess = 1, sample.size = 1)
slope_prior <- SdPrior(sigma.guess = 1, sample.size = 1)
seasonal_prior <- SdPrior(sigma.guess = 1, sample.size = 1)

# Add components
ss <- AddLocalLinearTrend(list(), y = train, level.sigma.prior = level_prior,
                          slope.sigma.prior = slope_prior)
ss <- AddSeasonal(ss, y = train, nseasons = 5)

# Fit model
bsts_model <- bsts(train, state.specification = ss, niter = 1500)
burn_in <- 500
bsts_pred <- predict(bsts_model, horizon = length(test), burn = burn_in)


# ARIMA Model
arima_model <- auto.arima(train, stepwise = FALSE, approximation = FALSE)
arima_pred <- forecast(arima_model, h = length(test))

# Evaluation Metrics
calculate_metrics <- function(pred, actual, model_type = "bsts") {
  if (model_type == "bsts") {
    pred_mean <- as.numeric(pred$mean)
    lower <- as.numeric(pred$interval[1, ])
    upper <- as.numeric(pred$interval[2, ])
  } else {
    pred_mean <- as.numeric(pred$mean)
    lower <- as.numeric(pred$lower[, "95%"])
    upper <- as.numeric(pred$upper[, "95%"])
  }
  
  list(
    MAE = mean(abs(pred_mean - actual)),
    RMSE = sqrt(mean((pred_mean - actual)^2)),
    Coverage = mean(actual >= lower & actual <= upper)
  )
}

bsts_metrics <- calculate_metrics(bsts_pred, test, "bsts")
arima_metrics <- calculate_metrics(arima_pred, test, "arima")

# Combine results
results <- data.frame(
  Model = c("BSTS", "ARIMA"),
  MAE = c(bsts_metrics$MAE, arima_metrics$MAE),
  RMSE = c(bsts_metrics$RMSE, arima_metrics$RMSE),
  Coverage = c(bsts_metrics$Coverage, arima_metrics$Coverage)
)
print(results)

# Convert predictions to time series
bsts_pred_ts <- ts(bsts_pred$mean,
                   start = c(1, train_size + 1),
                   frequency = 252)
arima_pred_ts <- ts(arima_pred$mean,
                    start = c(1, train_size + 1),
                    frequency = 252)
full_ts <- ts(y, frequency = 252)

# Plotting actual and predicted
autoplot(full_ts) +
  autolayer(bsts_pred_ts, series = "BSTS", color = "blue") +
  autolayer(arima_pred_ts, series = "ARIMA", color = "red") +
  xlab("Time") + ylab("TSLA Close Price") +
  ggtitle("TSLA Closing Price Forecast: BSTS vs ARIMA") +
  guides(colour = guide_legend(title = "Model"))

# Calibration plot for BSTS model
actuals <- test
pred_mean <- bsts_pred$mean
pred_lower <- bsts_pred$interval[1, ]
pred_upper <- bsts_pred$interval[2, ]
in_interval <- actuals >= pred_lower & actuals <= pred_upper

calibration_df <- data.frame(
  Time = 1:length(test),
  Actual = actuals,
  Predicted = pred_mean,
  Lower = pred_lower,
  Upper = pred_upper,
  InInterval = in_interval
)

ggplot(calibration_df, aes(x = Time)) +
  geom_ribbon(aes(ymin = Lower, ymax = Upper), fill = "skyblue", alpha = 0.4) +
  geom_line(aes(y = Predicted), color = "blue", size = 1) +
  geom_point(aes(y = Actual, color = InInterval), size = 1.5) +
  scale_color_manual(values = c("red", "black"), labels = c("Outside", "Inside")) +
  labs(title = "Calibration Plot for BSTS",
       y = "Closing Price", color = "95% Interval") +
  theme_minimal()